Load Data Check Data whether is null or not

In [16]:
!pip install cybotrade_datasource
!pip install python-dotenv

import pandas as pd
import numpy as np
import asyncio
import cybotrade_datasource
from datetime import datetime, timezone
from dotenv import load_dotenv
import os

import warnings as wr
wr.filterwarnings('ignore')

print(dir(cybotrade_datasource))

load_dotenv()

API_KEY = os.getenv('CYBOTRADE_API_KEY')

'''
async def main():
    data = await cybotrade_datasource.query_paginated(
        api_key=API_KEY, 
        topic='cryptoquant|btc/inter-entity-flows/miner-to-miner?from_miner=f2pool&to_miner=all_miner&window=hour', 
        start_time=datetime(year=2024, month=1, day=1, tzinfo=timezone.utc),
        end_time=datetime(year=2025, month=1, day=1, tzinfo=timezone.utc)
    )
    df = pd.DataFrame(data)
    print(df)
    

await main()
'''

'''
async def main():
    stream = await cybotrade_datasource.stream(
        api_key=API_KEY,
        topics=[
            'cryptoquant|btc/inter-entity-flows/miner-to-miner?from_miner=f2pool&to_miner=all_miner&window=hour',
            'cryptoquant|btc/market-data/liquidations?exchange=deribit&window=min',
        ],
    )
    async for msg in stream:
        print(msg)
    

await main()
'''

global_df = None 

async def get_data(topic):
    data = await cybotrade_datasource.query_paginated(
        api_key=API_KEY, 
        topic=topic, 
        limit=10000
    )
    return pd.DataFrame(data)


async def main():
    global global_df

    topics = [
        "bybit-linear|candle?symbol=BTCUSDT&interval=1m",
    ]

#topic
#"cryptoquant|btc/market-data/open-interest?exchange=bybit&window=min",
#bybit-linear|candle?interval=1m&symbol=BTCUSDT
#"cryptoquant|btc/market-data/coinbase-premium-index?window=hour",
#"cryptoquant|btc/exchange-flows/outflow?exchange=coinbase_advanced&window=hour",
#"glassnode|market/deltacap_usd?a=BTC&i=10m",

    tasks = [get_data(topic) for topic in topics]
    dataframes = await asyncio.gather(*tasks)

    for i, df in enumerate(dataframes):
        print(f"DataFrame {i+1} ({topics[i]})")
        print(df.head)
        print(df.isnull().sum())
        global_df = df

await main()


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'cybotrade_datasource', 'query', 'query_paginated', 'stream']
DataFrame 1 (bybit-linear|candle?symbol=BTCUSDT&interval=1m)
<bound method NDFrame.head of          start_time    close     high      low     open   volume
0     1743661080000  83221.0  83221.0  83172.5  83172.5   19.973
1     1743661140000  83237.9  83241.7  83221.0  83221.0   14.043
2     1743661200000  83200.0  83267.5  83200.0  83237.9  116.382
3     1743661260000  83267.5  83267.5  83200.0  83200.0   23.131
4     1743661320000  83304.5  83350.0  83267.4  83267.5   35.424
...             ...      ...      ...      ...      ...      ...
9995  1744260780000  81913.0  81921.6  81887.2  81921.5   25.376
9996  1744260840000  81943.3  81969.5  81913.0  81913.0   34.363
9997  1744260900000  81940.5  81955.6  81932.0  81943.3   34.894
9998  1744260960000  82000.1  82079.6  81940.5  81940.5  191.739
9999  1744261

Display first five column of the dataframe

In [17]:
print(global_df)

         start_time    close     high      low     open   volume
0     1743661080000  83221.0  83221.0  83172.5  83172.5   19.973
1     1743661140000  83237.9  83241.7  83221.0  83221.0   14.043
2     1743661200000  83200.0  83267.5  83200.0  83237.9  116.382
3     1743661260000  83267.5  83267.5  83200.0  83200.0   23.131
4     1743661320000  83304.5  83350.0  83267.4  83267.5   35.424
...             ...      ...      ...      ...      ...      ...
9995  1744260780000  81913.0  81921.6  81887.2  81921.5   25.376
9996  1744260840000  81943.3  81969.5  81913.0  81913.0   34.363
9997  1744260900000  81940.5  81955.6  81932.0  81943.3   34.894
9998  1744260960000  82000.1  82079.6  81940.5  81940.5  191.739
9999  1744261020000  81974.9  82000.2  81957.8  82000.1   25.968

[10000 rows x 6 columns]


Analyze Data (first five and last five)

In [18]:
global_df.head()

,start_time,close,high,low,open,volume
0,1743661080000,83221.0,83221.0,83172.5,83172.5,19.973
1,1743661140000,83237.9,83241.7,83221.0,83221.0,14.043
2,1743661200000,83200.0,83267.5,83200.0,83237.9,116.382
3,1743661260000,83267.5,83267.5,83200.0,83200.0,23.131
4,1743661320000,83304.5,83350.0,83267.4,83267.5,35.424


In [19]:
global_df.tail()

,start_time,close,high,low,open,volume
9995,1744260780000,81913.0,81921.6,81887.2,81921.5,25.376
9996,1744260840000,81943.3,81969.5,81913.0,81913.0,34.363
9997,1744260900000,81940.5,81955.6,81932.0,81943.3,34.894
9998,1744260960000,82000.1,82079.6,81940.5,81940.5,191.739
9999,1744261020000,81974.9,82000.2,81957.8,82000.1,25.968


In [20]:
global_df.shape

(10000, 6)

In [21]:
num_rows = len(global_df)
print(num_rows)

10000


We had 10000 rows in this dataset (latest)

In [22]:
cat_cols = global_df.select_dtypes(include=['object']).columns
num_cols = global_df.select_dtypes(include=np.number).columns.tolist()
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)

Categorical Variables:
Index([], dtype='object')
Numerical Variables:
['start_time', 'close', 'high', 'low', 'open', 'volume']


In [23]:
global_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   start_time  10000 non-null  int64  
 1   close       10000 non-null  float64
 2   high        10000 non-null  float64
 3   low         10000 non-null  float64
 4   open        10000 non-null  float64
 5   volume      10000 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 468.9 KB


Obtain a statistical summary of the dataframe using describe

In [24]:
global_df.describe()

,start_time,close,high,low,open,volume
count,1.000000e+04,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.743961e+12,80826.77586,80875.515700,80779.621850,80826.895620,117.012722
std,1.732137e+08,2705.14478,2687.784589,2722.349706,2705.222122,205.516421
min,1.743661e+12,74584.20000,74702.500000,74456.200000,74584.200000,0.946000
25%,1.743811e+12,78393.05000,78466.475000,78304.550000,78393.050000,25.769500
50%,1.743961e+12,82133.95000,82187.000000,82063.450000,82134.150000,58.804500
75%,1.744111e+12,83100.00000,83139.400000,83067.725000,83100.025000,129.974000
max,1.744261e+12,84670.20000,84736.000000,84656.100000,84670.200000,4595.508000


In [26]:
global_df.describe(include='all')

,start_time,close,high,low,open,volume
count,1.000000e+04,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.743961e+12,80826.77586,80875.515700,80779.621850,80826.895620,117.012722
std,1.732137e+08,2705.14478,2687.784589,2722.349706,2705.222122,205.516421
min,1.743661e+12,74584.20000,74702.500000,74456.200000,74584.200000,0.946000
25%,1.743811e+12,78393.05000,78466.475000,78304.550000,78393.050000,25.769500
50%,1.743961e+12,82133.95000,82187.000000,82063.450000,82134.150000,58.804500
75%,1.744111e+12,83100.00000,83139.400000,83067.725000,83100.025000,129.974000
max,1.744261e+12,84670.20000,84736.000000,84656.100000,84670.200000,4595.508000


In [27]:
num_rows = global_df.shape[0]
num_cols = global_df.shape[1]
print("Rows: ", num_rows, "Columns: ", num_cols)


Rows:  10000 Columns:  6


Show columns's data types using the property df.dtypes

In [28]:
global_df.dtypes

start_time      int64
close         float64
high          float64
low           float64
open          float64
volume        float64
dtype: object

In [29]:
summary_stats = global_df.describe()
print(summary_stats)

         start_time        close          high           low          open  \
count  1.000000e+04  10000.00000  10000.000000  10000.000000  10000.000000   
mean   1.743961e+12  80826.77586  80875.515700  80779.621850  80826.895620   
std    1.732137e+08   2705.14478   2687.784589   2722.349706   2705.222122   
min    1.743661e+12  74584.20000  74702.500000  74456.200000  74584.200000   
25%    1.743811e+12  78393.05000  78466.475000  78304.550000  78393.050000   
50%    1.743961e+12  82133.95000  82187.000000  82063.450000  82134.150000   
75%    1.744111e+12  83100.00000  83139.400000  83067.725000  83100.025000   
max    1.744261e+12  84670.20000  84736.000000  84656.100000  84670.200000   

             volume  
count  10000.000000  
mean     117.012722  
std      205.516421  
min        0.946000  
25%       25.769500  
50%       58.804500  
75%      129.974000  
max     4595.508000  
